# CREST and conformational searches
[CREST](https://crest-lab.github.io/crest-docs/) (originally abbreviated from Conformer-Rotamer Ensemble Sampling Tool) is a program for the automated exploration of the low-energy molecular chemical space. It functions as an OMP scheduler for calculations with efficient force-field and semiempirical quantum mechanical methods such as xTB, and provides a variety of capabilities for creation and analysis of structure ensembles.

First, let's take a look at what version of CREST you are working with. Paying attention to program versions is critical for a few reasons:
1. So you know what documentation to follow, because programs change! In fact, there are structured guidelines for [how Python program changes are reflected in version schemes](https://py-pkgs.org/07-releasing-versioning.html).
2. If you are using multiple computing resources for the same project you want to make sure the version is the same, in case differences across versions can alter results (hint, use virtual environments to contain speficic versions!)
3. Reproducibility! It is an absolute MUST to report what version of software you used in publications (even if the journal does not explicitly request it, this is a community standard). 
4. Can you think of any other reasons?

In [3]:
  ! crest --version | grep "Version"

       Version 2.12,   Thu 19. Mai 16:32:32 CEST 2022


<div class="alert alert-block alert-warning">
<b>CREST check</b>
    
Is anyone unable to get CREST to run? 

Do we have others with working CREST who are willing to share their laptop screen? </div>

In [6]:
! crest -h


       |                                            |
       |                 C R E S T                  |
       |                                            |
       |  Conformer-Rotamer Ensemble Sampling Tool  |
       |          based on the GFN methods          |
       |             P.Pracht, S.Grimme             |
       |          Universitaet Bonn, MCTC           |
       Version 2.12,   Thu 19. Mai 16:32:32 CEST 2022
  Using the xTB program. Compatible with xTB version 6.4.0

   Cite work conducted with this code as

   • P.Pracht, F.Bohle, S.Grimme, PCCP, 2020, 22, 7169-7192.
   • S.Grimme, JCTC, 2019, 15, 2847-2862.

   and for works involving QCG as

   • S.Spicher, C.Plett, P.Pracht, A.Hansen, S.Grimme,
     JCTC, 2022, 18 (5), 3174-3189.

   with help from:
   C.Bannwarth, F.Bohle, S.Ehlert, S.Grimme,
   C.Plett, P.Pracht, S.Spicher

   This program is distributed in the hope that it will be useful,
   but WITHOUT ANY WARRANTY; without even the implied warranty of
   M

In [50]:
! crest benzene.xyz --scratch benzene >> benzene.out

In [47]:
! crest benzene.xyz --quick --scratch benzene_crude >> benzene_crude.out

In [ ]:
! crest benzene.xyz --scratch benzene_crude2 --quick >> benzene_crude2.out

In [21]:
! crest benzene.xyz --mquick --scratch benzene_mega_crude >> benzene_mega_crude.out

In [ ]:
# Run a conformer search with Crest.
# --noreftopo prevent a lot of initial crashes by not doing a topology check prior to the conformational search.

! crest xtbopt.xyz --gfn2 --alpb water --noreftopo --T 24

# --gfn2//gfnff this does sampling and opt with GFN-FF and singlepoint with GFN2-xTB for greater speeds.


# TS conformers search can be performed in this way by constraining the TS mode. 

# Example of a constrains file for CREST

text = "$constrain\n"
text += "force constant=0.05\n"
text += "distance: 1,2,1.85\n"
text += "$end"

open("CREST-1.inp", "w")
with open("CREST-1.inp","a") as f:
    f.write(text)
    
! crest xtbopt.xyz --cinp CREST-1.inp --gfn2 --alpb water --noreftopo --T 24 >> CREST-1.out

# Some usefull preset constrain arguments

# --cbonds will constraints all covalent bonds
# --cmetal constraints all M-X bonds
# --cheavy constraints all heavy atom bonds

# If constraining a large portion of the molecule it is recomanded to constrain atom this way instead:
# This for example will constrain atom 1 to 26 and only perform sampling on atom 27 to 41

text = "$constrain\n"
text += "force constant=0.05\n"
text += "atoms: 1-26\n"
text += "$metadyn"
text += "atoms: 27-41"
text += "$end"

open("CREST-2.inp", "w")
with open("CREST-2.inp","a") as f:
    f.write(text)
    
# We can also generate the constrain input file automatically with:

# ! crest xtbopt.xyz --constrain 1-26 --gfn2 --alpb water --noreftopo --T 24


# Using the protonate tool to generate optimized protomers (no conf search)

! crest xtbopt.xyz --protonate --T 24 >> protonate.out
! cat protonated.xyz

# You can specify another type of monoatomic ion (default is proton)
# For example use --swel ca2+ for dicationic calcium

! crest xtbopt.xyz --swel Na1+ --protonate --T 24 >> NaProtonate.out

! crest xtbopt.xyz --deprotonate --T 24 >> deprotonate.out

! crest xtbopt.xyz --tautomerize --T 24 >> tautomerize.out

! crest waters.xyz --nci --gfn2 --alpb water --noreftopo --T 24 >> crest-water.out

! mv crest_conformers.xyz waters_conformers.xyz

! crest protonated-Na.xyz --nci --gfn2 --alpb water --noreftopo --T 24 >> crest_glutamine-Na.out


! mv crest_conformers.xyz glutamine-Na_conformers.xyz

# A ensemble of conformers can be refined using the cregen function of crest

! crest xtbopt.xyz --cregen glutamine_conformers.xyz --ewin 3 

# --ewin (default 6kcal/mol)
# --rthr RMSD treshold (default 0.125 Angstrom)
# --ethr energy treshold between conformer pairs (default 0.05 kcal/mol)
# --nmr --eqv NMR equivalencies
# --temp (default is 298.15 K)
# --notopo disables topology checks

# Cregen can also cluster structures based on diherdral angles

! crest xtbopt.xyz --cregen glutamine_conformers.xyz --cluster

## Non covalent complex: benzene dimer

In [41]:
! crest benzene_dimer.xyz --mquick  --scratch benzene_dimer >> benzene_dimer.out

In [42]:
! crest benzene_dimer.xyz --nci --mquick --scratch benzene_dimer_nci >> benzene_dimer_nci.out